In [1]:
#IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import copy
import pickle
from scipy.stats import t
import seaborn as sns
from scipy import stats
import inspect
import pickle

# LOADS

In [2]:
def load():
    FAv1=np.load('FAs_v1-corrected.npy')
    FAv2=np.load('FAs_v2-corrected.npy')
    GMv1C=np.load('GMs_v1-no-thr_corrected.npy')
    GMv1=np.load('GMs_v1-no-thr_no-corrected.npy')
    GMv2C=np.load('GMs_v2-thr_corrected.npy')
    GMv2=np.load('GMs_v2-thr_no-corrected.npy')
    RS=np.load('RSs_corrected_abs.npy')
    RSnA=np.load('RSs_corrected_no-abs.npy')
    RSnC=np.load('RSs_no-corrected_abs.npy')
    RSnCnA=np.load('RSs_no-corrected_no-abs.npy')
    clinic=pd.read_csv('clinic.csv')
    adj_mat=[FAv1,FAv2,GMv1C,GMv1,GMv2C,GMv2,RS,RSnA,RSnC,RSnCnA]
    return clinic, adj_mat
adj_mat_str=['FAv1','FAv2','GMv1C','GMv1','GMv2C','GMv2','RS','RSnA','RSnC','RSnCnA']
nodes=pd.read_csv('noms_nodes.csv')

In [3]:
def clinicPandaFA(clinic, adj_mat, adj_mat_str, thrFA):
    for matrix in range(10):
        if matrix==1:
            graphs=[]
            for k in range(165):
                for i in range(76):
                    for j in range(76):
                        if adj_mat[matrix][k][i][j]<thrFA:
                            adj_mat[matrix][k][i][j]=0.

                G = nx.Graph()
                graphs.append(nx.Graph(adj_mat[matrix][k]))      
            clinic[adj_mat_str[matrix]]=copy.deepcopy(graphs)
    return clinic
def clinicPandaGM(clinic, adj_mat, adj_mat_str, thrGM):
    for matrix in range(10):

         
        if matrix==4:
            graphs=[]
            for k in range(165):
                for i in range(76):
                    for j in range(76):
                        if adj_mat[matrix][k][i][j]<thrGM:
                            adj_mat[matrix][k][i][j]=0.

                G = nx.Graph()
                graphs.append(nx.Graph(adj_mat[matrix][k]))
            clinic[adj_mat_str[matrix]]=copy.deepcopy(graphs)

    return clinic
def clinicPandaRS(clinic, adj_mat, adj_mat_str, thrRS):
    for matrix in range(10):
        if matrix==6:
            graphs=[]
            for k in range(165):
                for i in range(76):
                    for j in range(76):
                        if adj_mat[matrix][k][i][j]<thrRS:
                            adj_mat[matrix][k][i][j]=0.

                G = nx.Graph()
                graphs.append(nx.Graph(adj_mat[matrix][k]))
            clinic[adj_mat_str[matrix]]=copy.deepcopy(graphs)

    return clinic

# METRIC FUNCTIONS

In [4]:
def node_strength(G, node):
    """
    Node strength is defined as the sum of the strengths (or weights) of all
    of the nodes edges.

    Parameters
    ----------
    G: graph
       A networkx graph
    node: object
       A node in the networkx graph

    Returns
    -------
    node strength: (int, float)
    """
    output = 0.0
    for edge in G.edges(node):
        output += G.get_edge_data(*edge)['weight']
    return output

In [5]:
def mainFA(thrFA, metrics,  pvalue):
    clinic, adj_mat=load()
    p_passed=[]
    p_passed_all=[]
    p_passed_partial=[]
    clinic=clinicPandaFA(clinic, adj_mat, adj_mat_str, thrFA)
    print('FA')
    for metric in metrics:
        print(str(metric))
        metricsCalculation(clinic, 'FAv2', metric,pvalue,p_passed, p_passed_all,p_passed_partial)
    return(p_passed, p_passed_all, p_passed_partial)

def mainGM(thrGM, metrics, pvalue):
    clinic, adj_mat=load()
    p_passed=[]
    p_passed_all=[]
    p_passed_partial=[]
    clinic=clinicPandaGM(clinic, adj_mat, adj_mat_str,thrGM)
    print('GM')
    for metric in metrics:
        print(str(metric))
        metricsCalculation(clinic, 'GMv2C', metric,pvalue,p_passed, p_passed_all,p_passed_partial)
    return(p_passed, p_passed_all, p_passed_partial)

def mainRS(thrRS, metrics, pvalue):
    clinic, adj_mat=load()
    p_passed=[]
    p_passed_all=[]
    p_passed_partial=[]
    clinic=clinicPandaRS(clinic, adj_mat, adj_mat_str,thrRS )
    print('RS')
    for metric in metrics:
        print(str(metric))
        metricsCalculation(clinic, 'RS', metric,pvalue,p_passed, p_passed_all,p_passed_partial)
    return(p_passed, p_passed_all, p_passed_partial)

In [6]:
def metricsCalculation(clinic,matrix, metric, pvalue, p_passed, p_passed_all,p_passed_partial):
    sans=[[] for i in range(76)]
    malalts0=[[] for i in range(76)]
    malalts1=[[] for i in range(76)]
    malalts2=[[] for i in range(76)]
    if metric=='node_strength':
        for j in range(165):
            for i in range(76):
                strenght=node_strength(clinic[matrix].iloc[j], i)
                if clinic['mstype'].iloc[j]==-1:
                    sans[i].append(strenght)
                elif clinic['mstype'].iloc[j]==0:
                    malalts0[i].append(strenght)
                elif clinic['mstype'].iloc[j]==1:
                    malalts1[i].append(strenght)
                elif clinic['mstype'].iloc[j]==2:
                    malalts2[i].append(strenght)
        sansM = [np.array(x).mean() for x in sans]
        malalts0M = [np.array(x).mean() for x in malalts0]
        malalts1M = [np.array(x).mean() for x in malalts1]
        malalts2M = [np.array(x).mean() for x in malalts2]

        nodeCar = copy.deepcopy(nodes)
        nodeCar['sans']=sansM
        nodeCar['malalts0']=malalts0M
        nodeCar['malalts1']=malalts1M
        nodeCar['malalts2']=malalts2M
        types=[sans, malalts0, malalts1, malalts2]
        typesStr=['Sans', 'RRMS', 'SPMS', 'PPMS']
        for i in range(76):
            sup=nodeCar.iloc[i]
            #if sup['importance']==1:
            nodeName=sup['region_name']
            supList=[]
            supList.append(sup['sans'])
            supList.append(sup['malalts0'])
            supList.append(sup['malalts1'])
            supList.append(sup['malalts2'])

            ind = np.arange(4)
            #fig, ax = plt.subplots()
            sup2=pd.DataFrame(columns=['type', 'value'])
            for k in sans[i]:
                sup2=sup2.append({'type':'Sans','value': k},ignore_index=True)
            for k in malalts0[i]:
                sup2=sup2.append({'type':'RRMS','value': k},ignore_index=True)
            for k in malalts1[i]:
                sup2=sup2.append({'type':'SPMS','value': k},ignore_index=True)
            for k in malalts2[i]:
                sup2=sup2.append({'type':'PPMS','value': k},ignore_index=True)
            #rects1 = ax.plot(ind, supList, '-ok')
            #ax.set_ylabel('Node Centrality')
            #ax.set_title('Node '+nodeName)
            #ax.set_xticks(ind +0.02 / 2)
            #ax.set_xticklabels(('Sans', 'RRMS', 'SPMS', 'PPMS'))
            #sns.boxplot(x="type", y="value", data=sup2)
            s0=stats.ttest_ind(sans[i], malalts0[i])[1]
            s1=stats.ttest_ind(sans[i], malalts1[i])[1]
            s2=stats.ttest_ind(sans[i], malalts2[i])[1]
            m01=stats.ttest_ind(malalts0[i], malalts1[i])[1]
            m02=stats.ttest_ind(malalts0[i], malalts2[i])[1]
            m12=stats.ttest_ind(malalts1[i], malalts2[i])[1]

            stats_iteration=[s0, s1, s2, m01, m02, m12]
            all_in=True
            aux=0
            count=0
            for s in stats_iteration:
                if s>pvalue:
                    all_in=False
                else:
                    p_passed.append([copy.copy(matrix),nodeName, 'node_strength', aux,s])
                    count+=1
                aux+=1
            if all_in:
                p_passed_all.append([copy.copy(matrix),nodeName,'node_strength',copy.copy(stats_iteration)])
            elif count>1:
                p_passed_partial.append([copy.copy(matrix),nodeName,'node_strength',copy.copy(stats_iteration)])

            #plt.show()

    else:
        if 'weight' in inspect.getargspec(metric).args:
            for j in range(165): 
                dicti=metric(clinic[matrix].iloc[j],weight='wheight')
                if clinic['mstype'].iloc[j]==-1:
                    for i in range(76):
                        sans[i].append(dicti[i])
                elif clinic['mstype'].iloc[j]==0:
                    for i in range(76):
                        malalts0[i].append(dicti[i])
                elif clinic['mstype'].iloc[j]==1:
                    for i in range(76):
                        malalts1[i].append(dicti[i])
                elif clinic['mstype'].iloc[j]==2:
                    for i in range(76):
                        malalts2[i].append(dicti[i])
        else:
            for j in range(165): 
                dicti=metric(clinic[matrix].iloc[j])
                if clinic['mstype'].iloc[j]==-1:
                    for i in range(76):
                        sans[i].append(dicti[i])
                elif clinic['mstype'].iloc[j]==0:
                    for i in range(76):
                        malalts0[i].append(dicti[i])
                elif clinic['mstype'].iloc[j]==1:
                    for i in range(76):
                        malalts1[i].append(dicti[i])
                elif clinic['mstype'].iloc[j]==2:
                    for i in range(76):
                        malalts2[i].append(dicti[i])
        sansM = [np.array(x).mean() for x in sans]
        malalts0M = [np.array(x).mean() for x in malalts0]
        malalts1M = [np.array(x).mean() for x in malalts1]
        malalts2M = [np.array(x).mean() for x in malalts2]

        nodeCar = copy.deepcopy(nodes)
        nodeCar['sans']=sansM
        nodeCar['malalts0']=malalts0M
        nodeCar['malalts1']=malalts1M
        nodeCar['malalts2']=malalts2M
        types=[sans, malalts0, malalts1, malalts2]
        typesStr=['Sans', 'RRMS', 'SPMS', 'PPMS']
        for i in range(76):
            sup=nodeCar.iloc[i]
            #if sup['importance']==1:
            nodeName=sup['region_name']
            supList=[]
            supList.append(sup['sans'])
            supList.append(sup['malalts0'])
            supList.append(sup['malalts1'])
            supList.append(sup['malalts2'])

            ind = np.arange(4)
            #fig, ax = plt.subplots()
            sup2=pd.DataFrame(columns=['type', 'value'])
            for k in sans[i]:
                sup2=sup2.append({'type':'Sans','value': k},ignore_index=True)
            for k in malalts0[i]:
                sup2=sup2.append({'type':'RRMS','value': k},ignore_index=True)
            for k in malalts1[i]:
                sup2=sup2.append({'type':'SPMS','value': k},ignore_index=True)
            for k in malalts2[i]:
                sup2=sup2.append({'type':'PPMS','value': k},ignore_index=True)
            #rects1 = ax.plot(ind, supList, '-ok')
            #ax.set_ylabel(str(metric).split(' ')[1])
            #ax.set_title(str(metric).split(' ')[1] +' '+nodeName)
            #ax.set_xticks(ind +0.02 / 2)
            #ax.set_xticklabels(('Sans', 'RRMS', 'SPMS', 'PPMS'))
            
            #sns.boxplot(x="type", y="value", data=sup2)
            s0=stats.ttest_ind(sans[i], malalts0[i])[1]
            s1=stats.ttest_ind(sans[i], malalts1[i])[1]
            s2=stats.ttest_ind(sans[i], malalts2[i])[1]
            m01=stats.ttest_ind(malalts0[i], malalts1[i])[1]
            m02=stats.ttest_ind(malalts0[i], malalts2[i])[1]
            m12=stats.ttest_ind(malalts1[i], malalts2[i])[1]
            stats_iteration=[s0, s1, s2, m01, m02, m12]
            all_in=True
            aux=0
            count=0
            for s in stats_iteration:
                if s>pvalue or s==float("nan"):
                    all_in=False
                else:
                    p_passed.append([copy.copy(matrix),nodeName, copy.copy(str(metric).split(' ')[1]), aux,s])
                    count+=1
                aux+=1
            if all_in:
                p_passed_all.append([copy.copy(matrix),nodeName,copy.copy(str(metric).split(' ')[1]),copy.copy(stats_iteration)])
            elif count>1:
                p_passed_partial.append([copy.copy(matrix),nodeName,copy.copy(str(metric).split(' ')[1]),copy.copy(stats_iteration)])
            #plt.show()
            
    return p_passed, p_passed_all,p_passed_partial

In [7]:
pv=[0.05,0.01,0.001]
metrics=['node_strength',nx.degree_centrality,nx.closeness_centrality,nx.betweenness_centrality,nx.clustering,nx.square_clustering,nx.pagerank]
resultatsFA=[]
for i in range(3):
    resultatsFA.append([pv[i]])
    for thr in range(0, 20):
        pssd,psdd_all,psdd_partial=mainFA(thr/20,metrics,pv[i])
        resultatsFA[i].append([copy.deepcopy(pssd),copy.deepcopy(psdd_all),copy.deepcopy(psdd_partial)])
with open("resultatsFA.pkl", "wb") as fp:   #Pickling
    pickle.dump(resultatsFA, fp)

FA
node_strength
<function degree_centrality at 0x7f0ba6d59430>


<ipython-input-6-31a680406986>:83: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if 'weight' in inspect.getargspec(metric).args:


<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
FA
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
FA
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
FA
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0

In [8]:
pv=[0.05,0.01,0.001]
metrics=['node_strength',nx.degree_centrality,nx.closeness_centrality,nx.betweenness_centrality,nx.clustering,nx.square_clustering,nx.pagerank]
resultatsGM=[]
for i in range(3):
    resultatsGM.append([pv[i]])
    for thr in range(0, 20):
        pssd,psdd_all,psdd_partial=mainGM(thr/20,metrics, pv[i])
        resultatsGM[i].append([copy.deepcopy(pssd),copy.deepcopy(psdd_all),copy.deepcopy(psdd_partial)])
with open("resultatsGM.pkl", "wb") as fp:   #Pickling
    pickle.dump(resultatsGM, fp)

GM
node_strength
<function degree_centrality at 0x7f0ba6d59430>


<ipython-input-6-31a680406986>:83: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if 'weight' in inspect.getargspec(metric).args:


<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
GM
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
GM
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
GM
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0

In [9]:
pv=[0.05,0.01,0.001]
metrics=['node_strength',nx.degree_centrality,nx.closeness_centrality,nx.betweenness_centrality,nx.clustering,nx.square_clustering,nx.pagerank]
resultatsRS=[]
for i in range(3):
    resultatsRS.append([pv[i]])
    for thr in range(0, 20):
        pssd,psdd_all,psdd_partial=mainRS(thr/20,metrics,pv[i])
        resultatsRS[i].append([copy.deepcopy(pssd),copy.deepcopy(psdd_all),copy.deepcopy(psdd_partial)])
with open("resultatsRS.pkl", "wb") as fp:   #Pickling
    pickle.dump(resultatsRS, fp)

RS
node_strength
<function degree_centrality at 0x7f0ba6d59430>


<ipython-input-6-31a680406986>:83: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  if 'weight' in inspect.getargspec(metric).args:


<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
RS
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
RS
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0x7f0ba6d0fdc0>
<function square_clustering at 0x7f0ba6d0fee0>
<function pagerank at 0x7f0ba6c73310>
RS
node_strength
<function degree_centrality at 0x7f0ba6d59430>
<function closeness_centrality at 0x7f0ba6d4d550>
<function betweenness_centrality at 0x7f0ba6d45820>
<function clustering at 0